In [19]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Annotated
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [21]:
class ChatState(TypedDict):
    
    messages: Annotated[list[BaseMessage],add_messages]

In [ ]:
llm = ChatOpenAI()

def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']
    
    # send to llm
    response = llm.invoke(messages)
    
    # response store state
    return {'messages': [response]}

In [ ]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [ ]:
initial_state = {'messages': [HumanMessage(content="Hello! How are you?")]}

chatbot.invoke(initial_state)

In [ ]:
thread_id = '1'

while True:

    input_text = input("Type here: ")

    print('User :', input_text)

    if input_text.strip().lower() in ['exit', 'quit','bye']:
        print("Exiting the chatbot. Goodbye!")
        break       
    
    config ={'configurable':{'thread_id':thread_id}}
    response = chatbot.invoke({'messages': [HumanMessage(content=input_text)]}, config=config)

    print('AI :', response['messages'][-1].content)